# 0 - Librerías y variables

In [1]:
# Librerías
# ------------------------------------------------------------------------------
import os
import requests
import json

from dotenv import load_dotenv, dotenv_values
load_dotenv()

True

In [2]:
# Variables
# ------------------------------------------------------------------------------
env_vars = dotenv_values()
for key in env_vars.keys():
    print(key)

OPENAI_API_KEY
PROXYCURL_API_KEY
TAVILY_API_KEY
LANGCHAIN_TRACING_V2
LANGCHAIN_ENDPOINT
LANGCHAIN_API_KEY
LANGCHAIN_PROJECT


# 1 - LLM Chat

En este apartado se llama a la API de OpenAI directamente:

In [3]:
# Importar OpenAI
from openai import OpenAI

# Inicializar el cliente
client = OpenAI()

# Llamar al LLM
messages = [{"role": "user", "content": "¿Cuánto es 1+1?"}]
    
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0.0,
)

response.choices[0].message.content

'1+1 es igual a 2.'

Si quiero modificar el código por ejemplo para utilizar un modelo como LLaMA 3.1 desde Ollama:

In [4]:
# Define el endpoint local de Ollama
OLLAMA_URL = "http://localhost:11434/api/chat"

# Llamar al LLM
messages = [{"role": "user", "content": "¿Cuánto es 1+1?"}]

response = requests.post(OLLAMA_URL, json={
    "model": "llama3.1",
    "messages": messages,
    "temperature": 0.0,
    "stream": False
})

data = response.json()
print(data["message"]["content"])

La respuesta es dos.


Cada vez que quiero apuntar a un LLM diferente, tengo que modificar el código de forma sustancial.

# 2 - LangChain

LangChain es más un framework para construir sistemas que usan modelos de lenguaje (LLMs) que una simple librería, a continuación se muestra una tabla con los bloques principales del framework:

| **Bloque**                       | **Para qué sirve**                                                                           |
| -------------------------------- | -------------------------------------------------------------------------------------------- |
| **1. Modelos**                   | Generan texto o representaciones vectoriales (embeddings) a partir de texto                  |
| **2. Prompts**                   | Construyen entradas reutilizables, seguras y controladas para los modelos                    |
| **3. Output Parsers**            | Transforman la salida del modelo (texto) en estructuras útiles como JSON, listas u objetos   |
| **4. Memorias**                  | Guardan el historial o contexto entre interacciones (conversacionales o no)                  |
| **5. Tools y Agents**            | Ejecutan funciones externas y permiten que un agente decida dinámicamente qué hacer y cuándo |
| **6. Runnables**                 | Unifican cualquier componente ejecutable en un flujo modular y componible                    |
| **7. Retrievers y VectorStores** | Permiten búsquedas semánticas para recuperar información relevante desde grandes corpus      |
| **8. Chains**                    | Encadenan pasos fijos de procesamiento en flujos controlados (prompt → modelo → parseo)      |
| **9. Callbacks y Tracing**       | Monitorizan, trazan y depuran la ejecución para mejorar observabilidad y debugging           |
| **10. Loaders y Splitters**      | Cargan y fragmentan documentos largos para su posterior análisis o búsqueda                  |

## 2.1. - Modelos

En LangChain, los models (modelos) son los componentes que generan texto o responden a mensajes. Son la parte que realmente interactúa con modelos de lenguaje (LLMs) como los de OpenAI, Anthropic, Cohere, etc. LangChain organiza los modelos según lo que hacen. Los más comunes son:

| Tipo de modelo     | Qué hace                                                                    | Clase típica                                |
| ------------------ | --------------------------------------------------------------------------- | ------------------------------------------- |
| **LLM**            | Genera texto a partir de un *prompt plano*                                  | OpenAI, HuggingFaceHub                      |
| **ChatModel**      | Maneja conversaciones con roles (usuario, asistente, sistema)               | ChatOpenAI, ChatAnthropic                   |
| **EmbeddingModel** | Convierte texto en vectores (útiles para búsquedas o comparación semántica) | OpenAIEmbeddings, HuggingFaceEmbeddings     |


LangChain permite llamar a diferentes modelos, con diferentes APIs, de forma agnóstica. Cambiando ligeramente el código, puedo aprovechar una estructura ya creada para apuntar a otro modelo.

In [5]:
import langchain
print(langchain.__version__)

0.3.25


In [6]:
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

In [7]:
# Crear los mensajes en el formato de LangChain
messages = [
    HumanMessage(content="¿Cuánto es 1+1?")
]

In [8]:
# Llamar a un modelo A
llm = ChatOllama(
    model="llama3.1",
    temperature=0.0
    )

response = llm.invoke(messages)

print(response.content)

La respuesta a la pregunta de "¿Cuánto es 1+1?" es 2.


In [9]:
# Llamar a un modelo B
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo", 
    temperature=0.0
    )

response = llm.invoke(messages)

print(response.content)

1+1 es igual a 2.


La estructura es exactamente la misma, se podría incluso encapsular el codigo en una función y pasar el modelo como parámetro.

In [10]:
def call_llm_model(
    model_name: str,
    temperature: float,
    message: str
) -> str:
    
    if model_name.startswith("gpt"):
        llm = ChatOpenAI(
            model_name=model_name,
            temperature=temperature
        )
    else:
        llm = ChatOllama(
            model=model_name,
            temperature=temperature
        )

    response = llm.invoke([HumanMessage(content=message)])
    return response.content


# Ejemplo de uso:
respuesta = call_llm_model(
    model_name="gpt-3.5-turbo",
    temperature=0.5,
    message="¿Quién es Andrew Ng?"
)

print(respuesta)

Andrew Ng es un científico e investigador en inteligencia artificial y aprendizaje profundo. Es cofundador de Google Brain y Coursera, y ha ocupado puestos de liderazgo en empresas como Baidu y Stanford University. Es conocido por sus contribuciones al campo de la inteligencia artificial y por su trabajo en la popularización de la educación en línea a través de cursos masivos abiertos en línea (MOOCs).


En las últimas versiones de LangChain, se ha creado la abstracción `init_chat_model()` para inicilizar modelos de diferentes proveedores desde la misma función.

In [11]:
from langchain.chat_models import init_chat_model

# model = init_chat_model("gpt-4o-mini", model_provider="openai")
# model = init_chat_model("claude-3-5-sonnet-latest", model_provider="anthropic")
# model = init_chat_model("mistral-large-latest", model_provider="mistralai")

## 2.2. - Prompt Templates

Las Prompt Templates (plantillas de prompt) son una herramienta de LangChain que te ayuda a crear automáticamente los mensajes que le envías al modelo de lenguaje, de una forma organizada, flexible y reutilizable.

Cuando trabajas con modelos como GPT, no sueles enviar directamente el texto del usuario al modelo. Normalmente quieres hacer algo más, como:

- Agregar instrucciones específicas (ej.: "Traduce al francés...")
- Dar un contexto adicional al modelo (ej.: "Eres un traductor especializado en literatura clásica...")
- Formatear el texto de forma especial (ej.: "Devuelve el resultado en un .json con la estructura...")
- Usar el mismo formato muchas veces con diferentes datos

Ahí es donde entran las prompt templates.

Piensa en ellas como plantillas con huecos (como los de un formulario). Tú defines una estructura fija y dejas espacios para completar con datos reales más tarde.

In [12]:
template = "Traduce el siguiente texto al {language}: {text}"

LangChain te permite definir esta plantilla y luego rellenarla automáticamente con los valores que quieras:
- `language = "francés"`
- `text = "Hola, ¿cómo estás?"`

Los principales templates y su propósito son:

| Prompt Template                    | ¿Para qué sirve?                                                                                                                                   |
| ---------------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------- |
| `ChatPromptTemplate`               | Plantilla para **modelos de chat**. Permite combinar mensajes (`System`, `Human`, etc.) fácilmente. Ideal para `ChatOpenAI`, `ChatAnthropic`, etc. |
| `PromptTemplate`                   | Plantilla simple de texto plano. Usado con modelos de lenguaje **no conversacionales** como `OpenAI(model="text-davinci-003")`.                    |
| `SystemMessagePromptTemplate`      | Subplantilla usada dentro de `ChatPromptTemplate` para definir el **mensaje del sistema**.                                                         |
| `HumanMessagePromptTemplate`       | Subplantilla usada dentro de `ChatPromptTemplate` para el **mensaje del usuario**                                                                  |
| `MessagesPlaceholder`              | Placeholder especial dentro de `ChatPromptTemplate` para insertar una lista dinámica de mensajes (ej: historial de conversación).                  |
| `AIMessagePromptTemplate`          | Subplantilla para simular un mensaje **anterior del asistente** en el historial.                                                                   |
| `FewShotPromptTemplate`            | Plantilla para tareas de **few-shot learning**. Permite definir ejemplos que se combinan con el input.                                             |
| `ChatMessagePromptTemplate`        | Versión más flexible que permite definir mensajes de un rol personalizado (ej: `"role": "function"`).                                              |
| `FewShotChatMessagePromptTemplate` | Lo mismo que `FewShotPromptTemplate`, pero adaptado para `ChatPromptTemplate`. Útil si quieres ejemplos en formato chat.                           |

In [13]:
from langchain_core.prompts import ChatPromptTemplate


| Método            | ¿Qué construye?                             | ¿Cuándo usarlo?                                 |
| ----------------- | ------------------------------------------- | ----------------------------------------------- |
| `from_template()` | Una sola plantilla de mensaje               | Para casos simples o rápidos                    |
| `from_messages()` | Una conversación entera con múltiples roles | Cuando necesitas dar contexto o varios mensajes |


In [14]:
# Plantilla de traducción
template = "Traduce el siguiente texto al {language}: {text}"

In [15]:
# Para casos sencillos
template_string = "Traduce el siguiente texto al {language}: {text}"
prompt_template = ChatPromptTemplate.from_template(template_string)

In [16]:
# Para casos mas complejos: sistema, usuarios, asistente...etc
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Eres un trabajador del campo, oriundo de la alpujarra de Granada."),
    ("human", "Traduce el siguiente texto al {language}: {text}")
])

In [17]:
prompt_template

ChatPromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Eres un trabajador del campo, oriundo de la alpujarra de Granada.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, template='Traduce el siguiente texto al {language}: {text}'), additional_kwargs={})])

In [18]:
prompt_template.messages[0]

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Eres un trabajador del campo, oriundo de la alpujarra de Granada.'), additional_kwargs={})

In [19]:
prompt_template.messages[1]

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, template='Traduce el siguiente texto al {language}: {text}'), additional_kwargs={})

In [20]:
prompt_template.messages[1].prompt.input_variables

['language', 'text']

In [21]:
language = "Granaíno"
text = "¿De dónde vienes?"

translation_prompt = prompt_template.format_messages(
    language=language,
    text=text
    )

In [22]:
translation_prompt

[SystemMessage(content='Eres un trabajador del campo, oriundo de la alpujarra de Granada.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Traduce el siguiente texto al Granaíno: ¿De dónde vienes?', additional_kwargs={}, response_metadata={})]

In [23]:
print(type(translation_prompt))
print(type(translation_prompt[0]))
print(type(translation_prompt[1]))

<class 'list'>
<class 'langchain_core.messages.system.SystemMessage'>
<class 'langchain_core.messages.human.HumanMessage'>


In [24]:
translator_response = llm.invoke(translation_prompt)
translator_response.content

'¿Ande vienes, majo?'

In [25]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Eres un noble de la realeza española del siglo XV."),
    ("human", "Traduce esto al {language}: {text}")
])

language = "Castellano antiguo"
text = "¿De dónde vienes?"

translation_prompt = prompt_template.format_messages(
    language=language,
    text=text
    )

customer_response = llm.invoke(translation_prompt)

customer_response.content

'¿De do vienes?'

## 2.3. - Output Parsers

En LangChain, los output parsers son herramientas clave para procesar y estructurar las respuestas que se obtienen de los modelos de lenguaje antes de ser utilizadas en otros pasos del flujo de trabajo. Estos parsers te permiten transformar la salida en bruto de los modelos en un formato más adecuado para tu aplicación, ya sea en forma de texto, datos estructurados o incluso en la ejecución de funciones específicas.

A continuación, una tabla con los output parsers más comunes en LangChain y su uso principal:

| **Output Parser**        | **¿Para qué sirve?**                                                                                                                                                 |
| ------------------------ | -------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `JsonOutputParser`       | Convierte la salida del modelo en un objeto JSON (estructura de diccionario o lista). Ideal cuando esperas respuestas estructuradas como JSON o diccionarios.        |
| `RegexOutputParser`      | Extrae información utilizando expresiones regulares. Se usa cuando la respuesta del modelo sigue un patrón específico que se puede identificar con regex.            |
| `StructuredOutputParser` | Permite parsear la salida en estructuras de datos más complejas. Ideal para cuando necesitas que el modelo devuelva datos tabulares o jerárquicos.                   |
| `VariableParser`         | Analiza las respuestas del modelo en busca de valores de variables específicas. Utilizado cuando deseas extraer datos de las respuestas para usarlos en otros pasos. |
| `TextOutputParser`       | Convierte la salida en un texto plano procesable. Ideal cuando la salida del modelo es simplemente texto sin estructura.                                             |
| `SQLOutputParser`        | Convierte la salida del modelo en consultas SQL estructuradas. Es útil si el modelo está generando consultas a bases de datos.                                       |
| `PythonOutputParser`     | Convierte la salida del modelo en código Python ejecutable. Utilizado cuando necesitas generar código a partir de la respuesta del modelo.                           |
| `ActionOutputParser`     | Permite que la salida sea convertida en una acción específica o una ejecución de función. Ideal para flujos de trabajo más dinámicos donde se deben ejecutar tareas. |

In [26]:
{
    "gift": False,
    "delivery_days": 5,
    "price_value": "muy asequible!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'muy asequible!'}

In [27]:
customer_review = """\
Este soplador de hojas es bastante increíble. Tiene cuatro configuraciones:\
soplador de vela, brisa suave, ciudad ventosa y tornado. \
Llegó en dos días, justo a tiempo para el regalo de aniversario de mi esposa. \
Creo que a mi esposa le gustó tanto que se quedó sin palabras. \
Hasta ahora he sido el único que lo ha usado, y lo he estado usando cada dos mañanas para limpiar las hojas de nuestro césped. \
Es ligeramente más caro que los otros sopladores de hojas que hay por ahí, pero creo que vale la pena por las características adicionales.
"""

review_template = """\
Para el siguiente texto, extrae la siguiente información:

gift: ¿Fue el artículo comprado como un regalo para otra persona? \
Responde True si es sí, False si no o si no se sabe.

delivery_days: ¿Cuántos días tardó en llegar el producto? \
Si esta información no se encuentra, devuelve -1.

price_value: Extrae cualquier frase sobre el valor o precio,\
y devuélvelas como una lista de Python separada por comas.

Formatea la salida como JSON con las siguientes claves:
gift --> boolean
delivery_days --> integer
price_value --> python list

texto: {text}
"""

In [28]:
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='Para el siguiente texto, extrae la siguiente información:\n\ngift: ¿Fue el artículo comprado como un regalo para otra persona? Responde True si es sí, False si no o si no se sabe.\n\ndelivery_days: ¿Cuántos días tardó en llegar el producto? Si esta información no se encuentra, devuelve -1.\n\nprice_value: Extrae cualquier frase sobre el valor o precio,y devuélvelas como una lista de Python separada por comas.\n\nFormatea la salida como JSON con las siguientes claves:\ngift --> boolean\ndelivery_days --> integer\nprice_value --> python list\n\ntexto: {text}\n'), additional_kwargs={})]


In [29]:
prompt_template.messages[0].prompt.input_variables

['text']

In [30]:
messages = prompt_template.format_messages(text=customer_review)

response = llm.invoke(messages)

print(response.content)

{
    "gift": true,
    "delivery_days": 2,
    "price_value": ["ligeramente más caro"]
}


In [31]:
type(response.content)

str

In [32]:
# Si intentamos acceder a response como si fuera un diccionario...
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

In [33]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [34]:
# Definir los esquemas de respuesta
response_schemas = [
    ResponseSchema(
        name="gift",
        description="True si fue un regalo, False si no."
    ),
    ResponseSchema(
        name="delivery_days",
        description="Número de días que tardó en llegar el producto."
    ),
    ResponseSchema(
        name="price_value",
        description="Frases relacionadas con el valor o precio del producto."
    )
]

# Crear el parser estructurado
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

print(output_parser)
print(output_parser.get_format_instructions())

response_schemas=[ResponseSchema(name='gift', description='True si fue un regalo, False si no.', type='string'), ResponseSchema(name='delivery_days', description='Número de días que tardó en llegar el producto.', type='string'), ResponseSchema(name='price_value', description='Frases relacionadas con el valor o precio del producto.', type='string')]
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // True si fue un regalo, False si no.
	"delivery_days": string  // Número de días que tardó en llegar el producto.
	"price_value": string  // Frases relacionadas con el valor o precio del producto.
}
```


Cuando se utiliza `response_schemas`, LangChain siempre interpreta que los resultados son string. Por eso en `review_template` se indica:

Formatea la salida como JSON con las siguientes claves:
- gift --> boolean
- delivery_days --> integer
- price_value --> python list

In [35]:
# Construir el prompt con instrucciones de formato
prompt_template = ChatPromptTemplate.from_template(review_template)

formatted_prompt = prompt_template.format(
    text=customer_review,
    format_instructions=output_parser.get_format_instructions()
)

formatted_prompt

'Human: Para el siguiente texto, extrae la siguiente información:\n\ngift: ¿Fue el artículo comprado como un regalo para otra persona? Responde True si es sí, False si no o si no se sabe.\n\ndelivery_days: ¿Cuántos días tardó en llegar el producto? Si esta información no se encuentra, devuelve -1.\n\nprice_value: Extrae cualquier frase sobre el valor o precio,y devuélvelas como una lista de Python separada por comas.\n\nFormatea la salida como JSON con las siguientes claves:\ngift --> boolean\ndelivery_days --> integer\nprice_value --> python list\n\ntexto: Este soplador de hojas es bastante increíble. Tiene cuatro configuraciones:soplador de vela, brisa suave, ciudad ventosa y tornado. Llegó en dos días, justo a tiempo para el regalo de aniversario de mi esposa. Creo que a mi esposa le gustó tanto que se quedó sin palabras. Hasta ahora he sido el único que lo ha usado, y lo he estado usando cada dos mañanas para limpiar las hojas de nuestro césped. Es ligeramente más caro que los otro

In [36]:
# Llamar al modelo
response = llm.invoke([HumanMessage(content=formatted_prompt)])

In [37]:
# Parsear la salida
parsed_output = output_parser.parse(response.content)

# Mostrar el resultado como diccionario
print(parsed_output)

{'gift': True, 'delivery_days': 2, 'price_value': ['ligeramente más caro']}


In [38]:
for key, value in parsed_output.items():
    print(f"Clave: {key} | Valor: {value} | Tipo de dato: {type(value)}")

Clave: gift | Valor: True | Tipo de dato: <class 'bool'>
Clave: delivery_days | Valor: 2 | Tipo de dato: <class 'int'>
Clave: price_value | Valor: ['ligeramente más caro'] | Tipo de dato: <class 'list'>


## 2.4. - LangChain: Memory

En LangChain, la memoria se refiere a la capacidad de una cadena o agente para recordar información entre interacciones. Es útil especialmente en contextos conversacionales o multistep, donde necesitas que el modelo tenga conocimiento del contexto anterior.

Por defecto, los modelos como GPT no tienen memoria entre llamadas. LangChain permite gestionar esto incluyendo “memoria” en una cadena (Chain) o un agente (Agent), haciendo que el contexto previo se incluya automáticamente en nuevas solicitudes.

A continuación, una tabla con los tipos de memoria más comunes en LangChain y su uso principal:

| **Tipo de Memoria**              | **¿Para qué sirve?**                                                                                                                                    |
| -------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `ConversationBufferMemory`       | Guarda todo el historial de la conversación como una cadena de texto. Es la forma más simple de mantener el contexto completo de una conversación.      |
| `ConversationBufferWindowMemory` | Similar a `ConversationBufferMemory`, pero solo conserva las **últimas *k* interacciones** (ventana deslizante). Útil para limitar el contexto enviado. |
| `ConversationTokenBufferMemory`  | Guarda la conversación basándose en un **límite de tokens**, no de turnos. Conserva solo la parte del historial que cabe dentro de un número de tokens. |
| `ConversationSummaryMemory`      | Resume el historial de la conversación en un texto corto. Ideal para mantener contexto en conversaciones largas sin exceder el límite de tokens.        |

Diferencias clave:

| Memoria                          | ¿Recuerda todo?       | ¿Controla tamaño? | ¿Resume contenido? | ¿Ideal para...?                               |
| -------------------------------- | --------------------- | ----------------- | ------------------ | --------------------------------------------- |
| `ConversationBufferMemory`       | Sí                    | ❌                 | ❌                  | Conversaciones cortas o demostraciones        |
| `ConversationBufferWindowMemory` | No (solo últimas *k*) | ✅ (*k*)           | ❌                  | Chats donde solo importa el contexto reciente |
| `ConversationTokenBufferMemory`  | No                    | ✅ (*n* tokens)    | ❌                  | Ajustar a límites estrictos de tokens         |
| `ConversationSummaryMemory`      | No (resume todo)      | ✅                 | ✅                  | Chats largos sin perder el hilo               |



In [39]:
from langchain_openai import ChatOpenAI


from langchain.chains import ConversationChain
from langchain.memory import (
    ConversationBufferMemory,
    ConversationBufferWindowMemory,
    ConversationTokenBufferMemory,
    ConversationSummaryBufferMemory
)

from langchain.schema import HumanMessage
from langchain.prompts import PromptTemplate


A continuación vemos un ejemplo, de como los modelos no incorporan memoria de forma nativa:

In [40]:
# Instanciar modelo
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo", 
    temperature=0.0
    )

# Primera interacción
message = "Mi nombre es Guille."
response = llm.invoke([HumanMessage(content=message)])
print("Primera respuesta:")
print(response.content)

# Segunda interacción sin memoria
message = "¿Cuál es mi nombre?"
response = llm.invoke([HumanMessage(content=message)])
print("\nSegunda respuesta (sin memoria):")
print(response.content)

Primera respuesta:
¡Hola Guille! ¿En qué puedo ayudarte hoy?

Segunda respuesta (sin memoria):
Lo siento, pero no tengo la capacidad de saber tu nombre a menos que me lo digas. ¿Cuál es tu nombre?


Vamos a darle memoria:

In [41]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo", 
    temperature=0.0
    )

memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm=llm, 
    memory=memory,
    verbose=True
)

/tmp/ipykernel_156115/1346084784.py:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
/tmp/ipykernel_156115/1346084784.py:8: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(


In [42]:
conversation.predict(input="Hola, mi nombre es Guille")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hola, mi nombre es Guille
AI:

> Finished chain.


'¡Hola Guille! ¡Encantado de conocerte! ¿En qué puedo ayudarte hoy?'

In [43]:
conversation.predict(input="¿Cuánto es 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hola, mi nombre es Guille
AI: ¡Hola Guille! ¡Encantado de conocerte! ¿En qué puedo ayudarte hoy?
Human: ¿Cuánto es 1+1?
AI:

> Finished chain.


'1+1 es igual a 2. ¿Hay algo más en lo que pueda ayudarte, Guille?'

In [44]:
conversation.predict(input="¿Cuál es mi nombre?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hola, mi nombre es Guille
AI: ¡Hola Guille! ¡Encantado de conocerte! ¿En qué puedo ayudarte hoy?
Human: ¿Cuánto es 1+1?
AI: 1+1 es igual a 2. ¿Hay algo más en lo que pueda ayudarte, Guille?
Human: ¿Cuál es mi nombre?
AI:

> Finished chain.


'Tu nombre es Guille. ¿Hay algo más que te gustaría saber?'

In [45]:
print(memory.buffer)
print(memory.load_memory_variables({}))

Human: Hola, mi nombre es Guille
AI: ¡Hola Guille! ¡Encantado de conocerte! ¿En qué puedo ayudarte hoy?
Human: ¿Cuánto es 1+1?
AI: 1+1 es igual a 2. ¿Hay algo más en lo que pueda ayudarte, Guille?
Human: ¿Cuál es mi nombre?
AI: Tu nombre es Guille. ¿Hay algo más que te gustaría saber?
{'history': 'Human: Hola, mi nombre es Guille\nAI: ¡Hola Guille! ¡Encantado de conocerte! ¿En qué puedo ayudarte hoy?\nHuman: ¿Cuánto es 1+1?\nAI: 1+1 es igual a 2. ¿Hay algo más en lo que pueda ayudarte, Guille?\nHuman: ¿Cuál es mi nombre?\nAI: Tu nombre es Guille. ¿Hay algo más que te gustaría saber?'}


In [46]:
memory = ConversationBufferMemory()

memory.save_context(
    {"input": "Hola"}, 
    {"output": "¿Qué tal?"}
)

print(memory.buffer)

Human: Hola
AI: ¿Qué tal?


In [47]:
memory.save_context(
    {"input": "No mucho, aquí andamos"}, 
    {"output": "Guay"}
)

print(memory.buffer)
print(memory.load_memory_variables({}))

Human: Hola
AI: ¿Qué tal?
Human: No mucho, aquí andamos
AI: Guay
{'history': 'Human: Hola\nAI: ¿Qué tal?\nHuman: No mucho, aquí andamos\nAI: Guay'}


In [48]:
# Otras opciones de memoria
# memory = ConversationBufferWindowMemory(k=1)
# memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)
# memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)

In [49]:

# Crear la memoria resumida (usa otro LLM para resumir, por defecto GPT-3.5)
summary_memory = ConversationSummaryBufferMemory(
    llm=ChatOpenAI(
        temperature=0,
        model_name="gpt-3.5-turbo",
    ),
    max_token_limit=50
)

# LLM para el chat (se puede utilizar el mismo LLM para ambas cosas, o no)
llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-3.5-turbo"
)

# Conversación con memoria
conversation = ConversationChain(
    llm=llm, 
    memory=summary_memory,
    verbose=True
)

# Interacciones
respuesta1 = conversation.predict(input="Hola, mi nombre es Guille, soy de Madrid y estoy aprendiendo a utilizar LangChain")
print("\n🗨️ Respuesta 1:")
print(respuesta1)

respuesta2 = conversation.predict(input="Trabajo como ingeniero de datos.")
print("\n🗨️ Respuesta 2:")
print(respuesta2)

respuesta3 = conversation.predict(input="¿Qué sabes de mí?")
print("\n🧠 Respuesta 3 (modelo usando resumen):")
print(respuesta3)

/tmp/ipykernel_156115/4275700198.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  summary_memory = ConversationSummaryBufferMemory(




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hola, mi nombre es Guille, soy de Madrid y estoy aprendiendo a utilizar LangChain
AI:

> Finished chain.

🗨️ Respuesta 1:
¡Hola Guille! ¡Qué gusto conocerte! Soy un asistente de inteligencia artificial diseñado para ayudarte con LangChain. ¿En qué puedo ayudarte hoy?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human introduces themselves as Guille from Madrid and mentions the

In [50]:
print(summary_memory.buffer)
print(summary_memory.load_memory_variables({}))

System: Guille from Madrid introduces themselves and mentions they are learning to use LangChain. The AI greets Guille and offers assistance with LangChain. Guille reveals they work as a data engineer. The AI acknowledges Guille's expertise in handling and analyzing large datasets and asks if they are seeking specific information about LangChain for their work. The AI summarizes Guille's background and offers to provide information about LangChain tailored to their expertise and needs.
{'history': "System: Guille from Madrid introduces themselves and mentions they are learning to use LangChain. The AI greets Guille and offers assistance with LangChain. Guille reveals they work as a data engineer. The AI acknowledges Guille's expertise in handling and analyzing large datasets and asks if they are seeking specific information about LangChain for their work. The AI summarizes Guille's background and offers to provide information about LangChain tailored to their expertise and needs."}


## 2.5. - Chains

En LangChain, una Chain (cadena) es una composición de pasos que conecta modelos de lenguaje con otras herramientas como funciones, bases de datos, prompts, parsers, memoria, etc. En lugar de lanzar una sola petición a un LLM, puedes encadenar operaciones de forma estructurada y reutilizable.

En otras palabras, una Chain permite definir flujos de trabajo con lógica.

A continuación, una tabla resumen de las Chains más comunes en LangChain y su uso principal:

| **Chain**               | **¿Para qué sirve?**                                                                                                                                              |
| ----------------------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `LLMChain`              | Es la cadena más simple. Conecta un `PromptTemplate` con un LLM. Ideal para tareas de entrada/salida básicas.                                                     |
| `SequentialChain`       | Permite ejecutar varias `Chains` de forma secuencial, pasando la salida de una como entrada de la siguiente. Útil para flujos paso a paso.                        |
| `SimpleSequentialChain` | Variante de `SequentialChain` más sencilla, que solo encadena directamente la salida de un paso como entrada al siguiente (sin nombres de variables intermedias). |
| `RouterChain`           | Redirige automáticamente la entrada a distintas sub-chains según su contenido. Ideal para sistemas multi-agente o flujos condicionados.                           |
| `ConversationChain`     | Incorpora memoria conversacional (como `BufferMemory`). Ideal para construir asistentes o chatbots con contexto.                                                  |
| `TransformChain`        | Aplica una transformación Python entre pasos. Útil para parsear, preprocesar o postprocesar datos entre modelos.                                                  |
| `MapReduceChain`        | Divide una tarea entre varios modelos (Map), luego une las respuestas (Reduce). Útil para resumir o analizar muchos documentos.                                   |
| `RetrievalQAChain`      | Combina un LLM con un buscador de documentos. Extrae contexto relevante antes de preguntar. Ideal para RAG.                                                       |

Las anteriores Chains son desarrolladas por LangChain, pero el usuario también puede definir sus propias chains encadenando diferentes modelos, templates, parsers...etc.

In [51]:
import pandas as pd

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain
from langchain.chains.router import MultiPromptChain

from langchain_openai import ChatOpenAI


In [52]:
data = {
    "nombre": ["EcoWave", "NeuroByte", "PetNanny", "Foodloop"],
    "sector": ["energía", "tecnología", "mascotas", "alimentación"]
}

df = pd.DataFrame(data)

df.head()

,nombre,sector
0,EcoWave,energía
1,NeuroByte,tecnología
2,PetNanny,mascotas
3,Foodloop,alimentación


In [53]:
# Instanciar modelo
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.9
)


### 2.5.1. - LLMChain

In [54]:
# Chain para generar eslogan
slogan_prompt = PromptTemplate.from_template(
    "Inventa un eslogan creativo para una startup llamada {nombre} del sector {sector}."
)

slogan_chain = LLMChain(
    llm=llm,
    prompt=slogan_prompt,
    output_key="eslogan"
)

slogan_chain.invoke({
    "nombre": df[['nombre']],
    "sector": df[['sector']]
})

/tmp/ipykernel_156115/1313909526.py:6: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  slogan_chain = LLMChain(


{'nombre':       nombre
 0    EcoWave
 1  NeuroByte
 2   PetNanny
 3   Foodloop,
 'sector':          sector
 0       energía
 1    tecnología
 2      mascotas
 3  alimentación,
 'eslogan': '1. EcoWave: "Haciendo olas verdes por un mundo más sostenible"\n2. NeuroByte: "Conectando mentes para un futuro brillante"\n3. PetNanny: "Cuidando a tu peludo como si fuera nuestro"\n4. Foodloop del sector energía: "Revolucionando la forma en que utilizamos la energía"\n5. Foodloop del sector tecnología: "Alimentando la innovación con cada bocado"\n6. Foodloop del sector mascotas: "Sabores que hacen ronronear a tu mejor amigo"\n7. Foodloop del sector alimentación: "Un ciclo de nutrición sostenible para todos"'}

### 2.5.2. - SimpleSequentialChain

Una SimpleSequentialChain es un conjunto de chains secuenciales, donde cada chain coge como input el output de la chain precedente sin indicar el nombre de los inputs/outputs de forma explícita. 

In [55]:
# Chain para generar eslogan
slogan_prompt = ChatPromptTemplate.from_template(
    "Inventa un eslogan creativo para una startup llamada {nombre}"
)

slogan_chain = LLMChain(llm=llm, prompt=slogan_prompt)


# Chain para campaña
campaign_prompt = ChatPromptTemplate.from_template(
    "Dado el eslogan {eslogan}, escribe una breve idea para una campaña publicitaria que transmita ese mensaje."
)

campaign_chain = LLMChain(llm=llm, prompt=campaign_prompt)


# Flujo combinado
overall_simple_chain = SimpleSequentialChain(
    chains=[slogan_chain, campaign_chain],
    verbose=True
)

In [56]:
overall_simple_chain.run(df[:1][['nombre']])

/tmp/ipykernel_156115/3648311525.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_simple_chain.run(df[:1][['nombre']])
The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Error in LangChainTracer.on_chain_start callback: ValueError('The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().')
Error in LangChainTracer.on_chain_start callback: ValueError('The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().')




> Entering new SimpleSequentialChain chain...
"Con EcoWave, haz olas de cambio eco-friendly en el mundo"
Una campaña publicitaria creativa para transmitir el mensaje de EcoWave podría incluir anuncios en redes sociales y sitios web destacando la importancia de hacer cambios sostenibles para proteger nuestro planeta. Se podrían utilizar imágenes impactantes de océanos limpios y playas libres de basura, junto con frases motivadoras como "Con EcoWave, juntos podemos crear un mundo más verde" o "Haz una ola de cambio eco-friendly con EcoWave". Además, se podrían organizar eventos de limpieza de playas y charlas educativas para involucrar a la comunidad en la causa.

> Finished chain.


'Una campaña publicitaria creativa para transmitir el mensaje de EcoWave podría incluir anuncios en redes sociales y sitios web destacando la importancia de hacer cambios sostenibles para proteger nuestro planeta. Se podrían utilizar imágenes impactantes de océanos limpios y playas libres de basura, junto con frases motivadoras como "Con EcoWave, juntos podemos crear un mundo más verde" o "Haz una ola de cambio eco-friendly con EcoWave". Además, se podrían organizar eventos de limpieza de playas y charlas educativas para involucrar a la comunidad en la causa.'

### 2.5.3. - SequentialChain

Similar a SimpleSequentialChain, pero indicando los nombres de inputs y outputs.

In [57]:
# Chain para generar eslogan
slogan_prompt = PromptTemplate.from_template(
    "Inventa un eslogan creativo para una startup llamada {nombre} del sector {sector}."
)

slogan_chain = LLMChain(
    llm=llm,
    prompt=slogan_prompt,
    output_key="eslogan"
)


# Chain para campaña
campaign_prompt = PromptTemplate.from_template(
    "Dado el eslogan {eslogan}, escribe una breve idea para una campaña publicitaria que transmita ese mensaje."
)

campaign_chain = LLMChain(
    llm=llm,
    prompt=campaign_prompt,
    output_key="campaña"
)

# Flujo combinado
sequential_chain = SequentialChain(
    chains=[slogan_chain, campaign_chain],
    input_variables=["nombre", "sector"],
    output_variables=["eslogan", "campaña"],
    verbose=True
)

In [58]:
# Lista para guardar resultados
resultados = []

# Aplicar la chain a cada fila
for _, fila in df.iterrows():

    entrada = {
        "nombre": fila["nombre"],
        "sector": fila["sector"]
    }

    salida = sequential_chain.invoke(entrada)
    
    resultados.append({
        **entrada,
        **salida  # esto añade 'eslogan' y 'campaña'
    })

# Nuevo DataFrame con resultados
df_resultado = pd.DataFrame(resultados)



> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


In [59]:
df_resultado.head()

,nombre,sector,eslogan,campaña
0,EcoWave,energía,"""Transformando olas en energía limpia para un ...",Imagínate un mundo donde las olas del mar no s...
1,NeuroByte,tecnología,"""Con NeuroByte, tu mente es el límite. ¡Conect...",La campaña publicitaria podría mostrar a difer...
2,PetNanny,mascotas,"""Tu mejor amigo está en buenas manos con PetNa...",La campaña publicitaria se centraría en mostra...
3,Foodloop,alimentación,"""¡Con Foodloop, cada bocado es un viaje de sab...",La campaña publicitaria podría consistir en un...


## 2.6. - Vector Stores y Retrievers

**Vector Store:**

Un vector store es una base de datos optimizada para almacenar y buscar vectores, que son representaciones numéricas de datos, como texto.

Cuando conviertes texto en vectores usando un modelo de embeddings, esos vectores se pueden almacenar en un vector store. Posteriormente, puedes hacer búsquedas por similitud: das un vector (por ejemplo, el de una pregunta del usuario) y el sistema devuelve los vectores más cercanos, que corresponden a los textos más relevantes.

Es la infraestructura que permite búsquedas semánticas rápidas y eficientes.

**Retrievers:**

Un retriever es un componente que, dado un texto de entrada (como una pregunta), recupera los documentos más relevantes desde alguna fuente. Es una abstracción de LangChain que encapsula la lógica de recuperación, sin preocuparse por cómo están almacenados los datos. El objetivo es simplemente: "dame lo más relevante que tengas sobre esta consulta".

Puede estar respaldado por un vector store, un motor de búsqueda clásico, una API externa, o cualquier otra fuente.

Cuando juntas un vector store con un retriever, lo que haces es usar el vector store como backend para que el retriever recupere documentos basándose en similitud semántica. De este modo se puede implementar un RAG, y darle un conocimiento extra al LLM.

In [60]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from langchain.chains import RetrievalQA
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch

from IPython.display import display, Markdown

In [61]:
# Loader de LangChain que lee el CSV y convierte cada fila en un documento
file = './data/OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [62]:
loader

In [63]:
docs = loader.load()
docs[:2]

[Document(metadata={'source': './data/OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries."),
 Document(metadata={'source': './data/OutdoorClothingCatalog_1000.csv', 'row': 1}, page_content=': 1\nname: Recycled 

In [64]:
# Instanciar embedding
embeddings = OpenAIEmbeddings()

# Convertir string a vector mediante un embedding
embed = embeddings.embed_query("Hola mi nombre es Guille")

# Visualizar embedding
print(len(embed))
print(embed[:5])

1536
[-0.04553837701678276, 0.014218204654753208, -0.010751419700682163, -0.01405520923435688, -0.018155166879296303]


In [65]:
# Crear VectorStore en memoria
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

/home/gmachin/.local/share/virtualenvs/exploring-langchain-UOzDUSui/lib/python3.11/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [66]:
# Query en lenguaje natural
query = "Please suggest a shirt with sunblocking"

# Llamar al VectorStore
docs_result = db.similarity_search(query, k=4)

# Documentos relevantes
print(len(docs_result))

4


In [67]:
docs_result[0]

Document(metadata={'source': './data/OutdoorClothingCatalog_1000.csv', 'row': 255}, page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\n\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\n\nSun Protection That Won\'t Wear Off\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.')

In [68]:
docs_result[1]

Document(metadata={'source': './data/OutdoorClothingCatalog_1000.csv', 'row': 374}, page_content=": 374\nname: Men's Plaid Tropic Shirt, Short-Sleeve\ndescription: Our Ultracomfortable sun protection is rated to UPF 50+, helping you stay cool and dry. Originally designed for fishing, this lightest hot-weather shirt offers UPF 50+ coverage and is great for extended travel. SunSmart technology blocks 98% of the sun's harmful UV rays, while the high-performance fabric is wrinkle-free and quickly evaporates perspiration. Made with 52% polyester and 48% nylon, this shirt is machine washable and dryable. Additional features include front and back cape venting, two front bellows pockets and an imported design. With UPF 50+ coverage, you can limit sun exposure and feel secure with the highest rated sun protection available.")

In [69]:
# Encadenar todas las respuestas en un string
qdocs = "".join([docs_result[i].page_content for i in range(len(docs_result))])

In [70]:
qdocs

': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\n\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\n\nSun Protection That Won\'t Wear Off\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.: 374\nname: Men\'s Plaid Tropic Shirt, Short-Sleeve\ndescription: Our Ultracomfortable sun protection is rated to UPF 50+, helping you stay cool and dry. Originally designed for fishing, this lightest hot-weather shirt offers UPF 50+ c

In [71]:
# Instancia LLM
llm = ChatOpenAI(
    temperature=0.0,
    model="gpt-3.5-turbo"
    )

# Prompt
prompt = f"""
I will give you a list of product entries. Each one starts with 'name:' and includes a 'description:'.
Extract all shirts with sun protection (UPF/UV) and return a markdown table with:

| Name | Summary of Sun Protection Features |

Here are the entries:

{qdocs}
"""

# Invocar al LLM con el prompt
response = llm.invoke(prompt) 

In [72]:
response.content

"| Name | Summary of Sun Protection Features |\n| --- | --- |\n| Sun Shield Shirt | UPF 50+ rated sun protection, blocks 98% of harmful UV rays |\n| Men's Plaid Tropic Shirt | UPF 50+ rated sun protection, blocks 98% of harmful UV rays |\n| Men's TropicVibe Shirt | UPF 50+ rated sun protection, blocks 98% of harmful UV rays |\n| Men's Tropical Plaid Short-Sleeve Shirt | UPF 50+ rated sun protection, blocks 98% of harmful UV rays |"

In [73]:
display(Markdown(response.content))

| Name | Summary of Sun Protection Features |
| --- | --- |
| Sun Shield Shirt | UPF 50+ rated sun protection, blocks 98% of harmful UV rays |
| Men's Plaid Tropic Shirt | UPF 50+ rated sun protection, blocks 98% of harmful UV rays |
| Men's TropicVibe Shirt | UPF 50+ rated sun protection, blocks 98% of harmful UV rays |
| Men's Tropical Plaid Short-Sleeve Shirt | UPF 50+ rated sun protection, blocks 98% of harmful UV rays |

In [74]:
# Convertir VectorStore en un retriever compatible con LangChain
retriever = db.as_retriever()

# RAG
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

query =  """
I will give you a list of product entries. Each one starts with 'name:' and includes a 'description:'.
Extract all shirts with sun protection (UPF/UV) and return a markdown table with:

| Name | Summary of Sun Protection Features |
"""

response = qa_stuff.run(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [75]:
display(Markdown(response))

| Name | Summary of Sun Protection Features |
| --- | --- |
| Sun Shield Shirt | SPF 50+ sun protection, blocks 98% of harmful rays |
| Men's Tropical Plaid Short-Sleeve Shirt | SPF 50+ sun protection, blocks 98% of harmful rays |
| Men's TropicVibe Shirt, Short-Sleeve | SPF 50+ sun protection, blocks 98% of harmful rays |
| Men's Plaid Tropic Shirt, Short-Sleeve | UPF 50+ sun protection, blocks 98% of harmful rays |

## 2.7. - Agents y Tools

**Tools:**

Una Tool en LangChain es una función externa que un modelo de lenguaje puede usar como si fuera una "extensión de sus capacidades". Las tools permiten que el LLM haga cosas que no puede hacer solo, como:
- Consultar una API externa (por ejemplo, una búsqueda en Google).
- Realizar cálculos matemáticos.
- Recuperar documentos con un retriever.
- Llamar a un sistema de base de datos, ejecutando SQL.

Cada Tool tiene tres partes básicas:
- Nombre
- Descripción (natural language prompt)
- Función ejecutable

Las tools son como comandos que el LLM puede invocar si se le entrena para reconocerlas y saber cuándo usarlas.

Importante: Las Tools por sí solas no hacen nada. Deben ser invocadas explícitamente, ya sea manualmente o por medio de un agente.


**Agents:**

Un Agent en LangChain es como un asistente inteligente que sabe tomar decisiones paso a paso. No solo genera texto como los modelos normales, sino que también puede pensar qué necesita hacer, elegir herramientas disponibles y usarlas para conseguir una respuesta completa. Es una de las piezas más avanzadas del framework.

El agente se comporta como un sistema reflexivo. Se le da un objetivo general (por ejemplo, “responde cualquier pregunta usando las herramientas disponibles”) y él decide, paso a paso:
- Qué información necesita para cumplir el objetivo.
- Qué Tool puede ayudar a conseguirla.
- Qué input debe darle a esa Tool.
- Qué hacer con la respuesta obtenida.
- Si necesita seguir usando otras Tools o ya puede generar la respuesta final.

Este proceso ocurre en bucles tipo:

```
Thought: Necesito buscar datos sobre X.
Action: Llamo a la Tool "GoogleSearch" con input "X"
Observation: La tool devuelve la info.
... (nuevo pensamiento)
Final Answer: Aquí está la respuesta completa.
```

Este patrón se llama "actuar → observar → razonar → actuar" y permite a los agentes resolver problemas complejos de forma iterativa, con autonomía limitada.

Por eso, los Agents no son simplemente una herramienta más, sino el orquestador que decide qué hacer, cuándo y cómo. Ellos ejecutan lógica de alto nivel usando las tools como extensiones de sus capacidades.

In [76]:
from langchain_openai import ChatOpenAI


from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools import PythonREPLTool


In [77]:
# Instancia LLM
llm = ChatOpenAI(
    temperature=0.0,
    model="gpt-3.5-turbo"
    )

# Instanciar tools
tools = load_tools(["llm-math","wikipedia"], llm=llm)

# Instanciar Agent
agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

/tmp/ipykernel_156115/2320382436.py:11: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent= initialize_agent(


In [78]:
# Ejemplo calculadora
agent.invoke("What is the 25% of 300?")



> Entering new AgentExecutor chain...
Thought: We can use a calculator to find 25% of 300.
Action:
```
{
  "action": "Calculator",
  "action_input": "25% of 300"
}
```
Observation: Answer: 75.0
Thought:Final Answer: 75.0

> Finished chain.


{'input': 'What is the 25% of 300?', 'output': '75.0'}

In [79]:
# Ejemplo wikipedia
agent.invoke("Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?")



> Entering new AgentExecutor chain...
Thought: I can use Wikipedia to find out which book Tom M. Mitchell wrote.
Action:
```
{
  "action": "wikipedia",
  "action_input": "Tom M. Mitchell"
}
```

/home/gmachin/.local/share/virtualenvs/exploring-langchain-UOzDUSui/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/gmachin/.local/share/virtualenvs/exploring-langchain-UOzDUSui/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past president of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed as the Interim Dean of the School of Computer Science at Carnegie Mellon.

Page: Tom Mitchell (Australian footballer)
Summary: Thomas Mitchell (born 31 May 1993) is a professional Australian rules footb

{'input': 'Tom M. Mitchell is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU)what book did he write?',
 'output': 'The book written by Tom M. Mitchell is "Machine Learning."'}

In [80]:
# Instanciar Agent
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

customer_list = [
    ["Harrison", "Chase"], 
    ["Lang", "Chain"],
    ["Dolly", "Too"],
    ["Elle", "Elem"], 
    ["Geoff","Fusion"], 
    ["Trance","Former"],
    ["Jen","Ayai"]
]

agent.invoke(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


We can use the `sorted()` function in Python to sort the list of customers based on last name first and then first name.
Action: Python_REPL
Action Input: sorted([['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']], key=lambda x: (x[1], x[0]))
Observation: 
Thought:The customers are now sorted by last name and then first name.
Final Answer: [['Jen', 'Ayai'], ['Harrison', 'Chase'], ['Lang', 'Chain'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Dolly', 'Too']]

> Finished chain.


{'input': "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
 'output': "[['Jen', 'Ayai'], ['Harrison', 'Chase'], ['Lang', 'Chain'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Dolly', 'Too']]"}

In [81]:
import langchain
langchain.debug=True

agent.invoke(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 

langchain.debug=False

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an

In [89]:
from datetime import date
from langchain.agents import tool


@tool
def time(text: str) -> str:
    """
    Returns todays date, use this for any questions related to knowing todays
    date.
    
    The input should always be an empty string, and this function will always
    return todays date - any date mathmatics should occur outside this function.
    """

    return str(date.today())


agent= initialize_agent(
    tools + [time], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)


agent.invoke("whats the date today?") 

ValueError: ChatAgent does not support multi-input tool time.